<h1 style="text-align: center"> Pima Indians Diabetes Database </h1>

<p><b><i> first i will do some explaratory data analysis to he data set to get some insights and visulizations</i></b></p>

<h4> data wrangling process </h4>
<ol>
    <li>data gathering </li>
    <li>data assesing </li>
    <li>data cleaing </li>
</ol>
<h6> then i will do some analysis </h6>
<h6> then i will make an classification model

In [ ]:
## importing some usfel data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline 

## data gathering 
- i uploaded it to my nootbok

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        data = os.path.join(dirname, filename)

df = pd.read_csv(data)
print('the data has {} column and {} rows'.format(df.shape[1] ,df.shape[0]))

## data assesing 
- here i will see the data to get guick overview about the records and know if there any problems 

#### visual assesment 

In [ ]:
df.head()

- <p style="color:red" > there is some 0 value in the features column and that doesn't make any sense </p>

#### visual assesment

In [ ]:
# check the data type
df.info()

- <p style="color:red" >all data types are correct but its clear the the null values represented with 0 not null </p>

In [ ]:
# check duplicate values 
if bool(df.duplicated().sum()) :
    print('ther is duplicate rows')
else :
    print('ther is no duplicate rows')

In [ ]:
df.describe()

### assesing report 
change the zeros values in <i>Glucose-BloodPressure-SkinThickness-Insulin-BMI</i> with the mean <br> 
<p style="text-align: center" ><b> this is the only problem in the data set that need to clean</b></p>

## data cleaning

###### define .
- there is 0 value in some column thet doesn't make any sense so i will convert them to the means value  

In [ ]:
(df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] == 0).sum()

###### code

In [ ]:
from sklearn.impute import SimpleImputer 
need_to_vonvert = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']]
ImputedModule = SimpleImputer(missing_values = 0 , strategy ='mean') 
Imputed = ImputedModule.fit(need_to_vonvert) 
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = Imputed.transform(need_to_vonvert) 
 

##### test

In [ ]:
(df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] == 0).sum()

In [ ]:
df.describe()

<h1 style="text-align: center; color:red"> EDA </h1>

In [ ]:
from matplotlib import rcParams
## see outcome counts
rcParams['figure.figsize'] =(8,6)
plt.pie(x=df['Outcome'].value_counts().values ,explode = [0.01,0.01] ,
            labels =['no diabetes', 'diabetes'] , startangle = 90, autopct= '%1.1f%%', counterclock = False);
plt.title('outcome' ,fontsize=18);

##### see the counts of the Pregnancies

In [ ]:
from matplotlib import rcParams
sns.set(style="whitegrid")
groupedvalues = df['Pregnancies'].value_counts()
g=sns.countplot(x="Pregnancies", data=df,linewidth=5 ,palette="Blues_d");
rcParams['figure.figsize'] =(15,10)
for index, row in zip(groupedvalues.index ,groupedvalues.values):
    g.text(index ,row, round(row,2), color='black', ha="center" , fontsize=15)
g.axes.get_yaxis().set_visible(False)
plt.xlabel('Pregnancies' , fontsize=18);
plt.title('the count of pregancies values' , fontsize=12);

##### the relation between age and diabetes

In [ ]:
sns.set()
rcParams['figure.figsize'] =(8,6)
sns.violinplot(data=df, x='Outcome', y='Age', color=sns.color_palette()[0] , inner='quartile' );
plt.xlabel('Outcome');
plt.ylabel('Age');
plt.title('Age distrubution' , fontsize=18);

#### see corelation 

In [ ]:
g = sns.set(style="ticks", color_codes=True)
g= sns.pairplot(df, hue="Outcome", palette="husl" );
handles = g._legend_data.values()
labels = g._legend_data.keys()
g.fig.legend(handles=handles, labels=labels, loc='upper center', ncol=1 , fontsize=18);
g.fig.subplots_adjust(top=0.92, bottom=0.08);


In [ ]:
corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool));
cmap = sns.diverging_palette(230, 20, as_cmap=True);
sns.heatmap(corr, mask=mask, cmap=cmap , linewidths=.5);
plt.title('correlation heatmap' , fontsize=18);

# classification model 

In [ ]:
#determine x , y
x = df.drop('Outcome' , axis=1)
y = df.Outcome

x.shape , y.shape

### data rescaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(copy=True, with_mean=True, with_std=True) 
x = scaler.fit_transform(x) 
print('X \n' , x[:2]) 
print('y \n' , y[:2].values)

### data splitting

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
x_poly = poly.fit_transform(x)



x_train, x_test, y_train, y_test = train_test_split(x_poly, y, test_size=0.30, random_state=44, shuffle =True) 
 
print('X_train shape is ' , x_train.shape) 
print('X_test shape is ' , x_test.shape) 
print('y_train shape is ' , y_train.shape) 
print('y_test shape is ' , y_test.shape) 

### logistic regression "classification"
<p style="color:red" > if i try neural_network  it will give similar results as the logistic regression model </p>

In [ ]:
from sklearn.linear_model import LogisticRegression 


LogisticRegressionModel = LogisticRegression(penalty='l2',solver='sag' , C=1.0, random_state=33  ) 
LogisticRegressionModel.fit(x_train, y_train) 
 
print('LogisticRegressionModel Train Score is : ' , LogisticRegressionModel.score(x_train, y_train)) 
print('LogisticRegressionModel Test Score is : ' , LogisticRegressionModel.score(x_test, y_test)) 
print('LogisticRegressionModel Classes are : ' , LogisticRegressionModel.classes_)


### calculate some metric about the test prediction 

In [ ]:
from sklearn.metrics import f1_score ,classification_report , plot_confusion_matrix

y_pred = LogisticRegressionModel.predict(x_test)
plot_confusion_matrix(LogisticRegressionModel, x_test, y_test ) ;
plt.show()
 
F1Score = f1_score(y_test, y_pred, average='micro')
print('F1 Score is : ', F1Score)

ClassificationReport = classification_report(y_test,y_pred) 
print('\nClassification Report is : ', ClassificationReport ) 
 


##### save the model 

In [ ]:
import joblib as jb
jb.dump(LogisticRegressionModel , 'saved file.sav') 
 

### try k. nearest Neighbors 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=44, shuffle =True) 

from sklearn.neighbors import KNeighborsClassifier 

KNNClassifierModel = KNeighborsClassifier(n_neighbors= 5,weights ='distance', algorithm='auto') 
KNNClassifierModel.fit(x_train, y_train)

print('KNNClassifierModel Train Score is : ' , KNNClassifierModel.score(x_train, y_train)) 
print('KNNClassifierModel Test Score is : ' , KNNClassifierModel.score(x_test, y_test)) 

y_pred = KNNClassifierModel.predict(x_test)
plot_confusion_matrix(KNNClassifierModel, x_test, y_test ) ;

 - it very good at the train set but at the test data the logistic regression is better

### trying descion tree 
<p style="color:red" ><b> iceasing max_depth will increase the accuracy but will increase the time too </b></p>

In [ ]:

from sklearn.tree import DecisionTreeClassifier

DecisionTreeClassifierModel = DecisionTreeClassifier(criterion='entropy',max_depth=10,random_state=33)
DecisionTreeClassifierModel.fit(x_train, y_train)

print('DecisionTreeRegressor Train Score is : ' , DecisionTreeClassifier.score(self = DecisionTreeClassifierModel , X= x_train, y= y_train) ) 
print('DecisionTreeRegressor Test Score is : ' ,DecisionTreeClassifier.score(self = DecisionTreeClassifierModel , X= x_test, y= y_test))